In [175]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")


In [117]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "../data/atomic-habits.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

content = []
for i in range(len(pages)):
    content.append(pages[i].page_content)



In [177]:
client = OpenAI(api_key=api_key)
cleaned_list = []
cleaning_prompt = '''Here's a prompt you can use with the OpenAI API to clean your text efficiently:
You are a text cleaning and formatting assistant. Your task is to clean and format the following text from a book summary, maintaining its original meaning and structure while improving readability. Please follow these guidelines: 
Remove any special characters or symbols that aren't standard punctuation.
Fix any spacing issues, ensuring single spaces between words and appropriate line breaks.
Correct obvious spelling errors and merge split words (e.g., 'habit s' to 'habits').
Format bullet points consistently using '-' as the bullet character.
Preserve the original structure of sections and subsections.
Capitalize the first letter of each sentence and proper nouns.
Ensure consistent formatting for titles and subtitles.
Retain the core content and meaning of the text.
Remove any emojis and non text stuff including asteriks.
Remove any new line characters like '\n' and words such as here'/s and it'/s, fix them and make here's and it's.

Please provide the cleaned and formatted version of this text.
Here's the text to clean and format:
'''

for item in content:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": cleaning_prompt},
                {"role": "user", "content": f"Clean and format the text:\n\n{item}"}
            ]
        )
        cleaned_text = completion.choices[0].message.content
        cleaned_list.append(cleaned_text)

# Print the response
print(completion.choices[0].message.content)

Here is the cleaned and formatted text:

Atomic Habits Notes

I have to admit that your life was transformed by one small change. The holy grail of habit change is not a single 1 percent improvement, but a thousand of them. It's a bunch of atomic habits stacking up, each one a fundamental unit of the overall system.

This is a continuous process. There is no finish line. There is no permanent solution. Whenever you're looking to improve, you can rotate through the Four Laws of Behavior Change until you find the next bottleneck. 

- Make it obvious. 
- Make it attractive. 
- Make it easy. 
- Make it satisfying. 

Round and round. Always looking for the next way to get 1 percent better. The secret to getting results that last is to never stop making improvements.


In [97]:
cleaned_list

['Here\'s the cleaned and formatted version of the text:\n\nAtomic Habits Notes\n\n1. Atomic Habits Notes\n\nThe first three laws of behavior change - make it obvious, make it attractive, and make it easy - increase the odds that a behavior will be performed this time. The fourth law of behavior change - make it satisfying - increases the odds that a behavior will be repeated next time. It completes the habit loop.\n\nFundamentals: Why Tiny Changes Make a Big Difference\n\nChapter 3: How to Build Better Habits in 4 Simple Steps\n\nDefinition of Habits\n\n- Habits: Automatic behaviors that solve recurring problems in our environment.\n- Quote: "Habits are, simply, reliable solutions to recurring problems in our environment."\n- Purpose: Save mental energy by automating responses to common situations.\n\nThe Habit Loop: 4-Part Pattern',
 'Cleaned and formatted text:\n\nAtomic Habits Notes\n\n- Cue: Trigger for your brain, tied to a reward\n- Craving: Transformation of the trigger into de

In [185]:
import re

def remove_new_lines_and_slashes(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\u2014', ' ', text)
    text = re.sub(r"\\'", "'", text)
    return text

cleaned_list = [remove_new_lines_and_slashes(item) for item in cleaned_list]

In [186]:
cleaned_list

['Formatted Text:  Atomic Habits Notes  1. Atomic Habits Notes  The first three laws of behavior change make it obvious, make it attractive, and make it easy increase the odds that a behavior will be performed this time. The fourth law of behavior change make it satisfying increases the odds that a behavior will be repeated next time. It completes the habit loop.  The Fundamentals: Why Tiny Changes Make a Big Difference  Chapter 3: How to Build Better Habits in 4 Simple Steps  Definition of Habits  Habits: Automatic behaviors that solve recurring problems in our environment  Quote: "Habits are simply reliable solutions to recurring problems in our environment."  Purpose: Save mental energy by automating responses to common situations  The Habit Loop: 4-Part Pattern',
 'Atomic Habits Notes  - Cue: Trigger for your brain, tied to a reward - Craving: Transformation of the trigger into desire - Response: Performing the habitual action - Reward: Positive feeling that reinforces the habit  P

In [187]:
import json

# This will create the file and write your data to it
with open('atomic_habits_notes.json', 'w') as f:
    json.dump({"notes": cleaned_list}, f, indent=2)

print("Notes saved to atomic_habits_notes.json")

Notes saved to atomic_habits_notes.json


In [191]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please resposne to the user request only based on the given context"),
        ("user","Question:{question}\nContext:{context}")


    ]


)
model=ChatOpenAI(model="gpt-3.5-turbo")
output_parser=StrOutputParser()

chain=prompt|model|output_parser
question="Can you summarize the speech?"
context=""" Even though you are only a very small speck of ocean, of the 300 million children on whose shoulders the future of India rests, you can ignite the minds, light the fire, become a burning candle to light another candle. My dear children, you work in the school for about 25000 hours before you complete 12th std. Within the school curriculum / school hours, I want you to contribute to the Mission of Developed India by involving in the student centric activities like Literacy Development, Eco-Care Movement.

India after its independence was determined to move ahead with planned policies for Science & Technology. Now, India is very near to self-sufficiency in food, making the ship to mouth existence of 1950s, an event of the past. Also improvements in the health sector, have eliminated few contagious diseases. There is a increase in life expectancy. Small scale industries provide high percentage of National GDP - a vast change in 1990s compared to 1950s. Today India can design, develop and launch world class geo-stationary and sun synchronous, remote sensing satellites.

The Nuclear establishments have reached the capability of building nuclear power stations, nuclear medicine and nuclear irradiation of agricultural seeds for growth in agricultural production. Today India has become a Nuclear Weapon State. Defence Research had led to design, development and production of Main Battle Tanks, strategic missile systems, electronic warfare systems and various armours. India is a missile power. Also we have seen growth in the Information Technology; the country is progressing in hardware and software export business of more than 10 billion dollars even though there are low ebbs in the last few years. India yet is a developing country. What Technology can do further?

Technology has multiple dimensions. Geopolitics convert the technology to a particular nation's policy. The same policy will lead to economic prosperity and capability for national security. For example, the developments in chemical engineering brought fertilizers for higher yield of crops while the same science led to chemical weapons. Likewise, rocket technology developed for atmospheric research helped in launching satellites for remote sensing and communication applications which are vital for the economic development. The same technology led to development of missiles with specific defense needs that provides security for the nation. The aviation technology development has led to fighter and bomber aircraft, and the same technology will lead to passenger jet and also help operations requiring quick reach of support to people affected by disasters. At this stage, let us study global growth of technology and impact in human life."""

print(chain.invoke({"question":question,"context":context}))

The speech emphasizes the importance of education and student involvement in activities that contribute to the development of India. It highlights India's advancements in science, technology, healthcare, and defense since gaining independence. It also discusses the role of technology in various aspects of national development and security. The speaker encourages students to ignite their minds and become active participants in the mission of a Developed India.
